In [1]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

# Load the tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("samirangupta31/MiniProject")
model = AutoModelForSequenceClassification.from_pretrained("samirangupta31/MiniProject")


tokenizer_config.json:   0%|          | 0.00/1.29k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.76k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/433M [00:00<?, ?B/s]

In [7]:
import pandas as  pd
df=pd.read_csv('/kaggle/input/medical-transcripts/transcript_clean.csv')
symptom_labels = df.prompt.unique()

evidence_dict = {}
for index, symptom_labels in enumerate(symptom_labels):
    evidence_dict[symptom_labels] = index
evidence_dict

{'Emotional pain': 0,
 'Back pain': 1,
 'Joint pain': 2,
 'Infected wound': 3,
 'Foot ache': 4,
 'Shoulder pain': 5,
 'Injury from sports': 6,
 'Skin issue': 7,
 'Stomach ache': 8,
 'Knee pain': 9,
 'Heart hurts': 10,
 'Hard to breath': 11,
 'Head ache': 12,
 'Body feels weak': 13,
 'Feeling dizzy': 14,
 'Open wound': 15,
 'Internal pain': 16,
 'Blurry vision': 17,
 'Acne': 18,
 'Muscle pain': 19,
 'Hair falling out': 20,
 'Neck pain': 21,
 'Cough': 22,
 'Ear ache': 23,
 'Feeling cold': 24}

In [2]:
import torch

def preprocess_input(text):
    encoding = tokenizer.encode_plus(
        text,
        add_special_tokens=True,
        max_length=512,
        padding='max_length',
        truncation=True,
        return_tensors='pt'
    )
    input_ids = encoding['input_ids']
    attention_mask = encoding['attention_mask']
    return input_ids, attention_mask

def predict_intent(input_text):
    input_ids, attention_mask = preprocess_input(input_text)
    with torch.no_grad():
        outputs = model(input_ids, attention_mask=attention_mask)
    logits = outputs.logits
    predicted_class = torch.argmax(logits, dim=1).item()
    return predicted_class


In [ ]:
import json

# Load intents data
with open('/kaggle/input/intent-dataset/intents.json') as f:
    data = json.load(f)
    
# Create a mapping from class index to response
index_to_response = {v: tg['responses'] for tg in data['intents'] for v in [evidence_dict[tg['tag']]]}

def chat():
    print("Start talking with the bot! Type 'quit' to exit.")
    while True:
        user_input = input("You: ")
        if user_input.lower() == "quit":
            break

        predicted_class = predict_intent(user_input)
        responses = index_to_response.get(predicted_class, ["Sorry, I didn't understand that."])
        print(f"Bot: {responses[0]}")

chat()


Start talking with the bot! Type 'quit' to exit.


You:  I am feeling pain in stomach


Bot: It seems like you may have Stomach ache. Please consult a doctor.


You:  I am feeling down and I don't feel lik doing anythin


Bot: It seems like you may have Emotional pain. Please consult a doctor.


You:  I don wan t fee lik eatin


Bot: It seems like you may have Internal pain. Please consult a doctor.
